<a href="https://colab.research.google.com/github/premgorivale1995/Image-Classification/blob/main/Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ipython-autotime #Using this it shows us execution time of cell.
%load_ext autotime

In [ ]:
!pip install bing-image-downloader  #installing Bing images

In [ ]:
!mkdir images

In [ ]:
# Data Gathering - 
from bing_image_downloader import downloader
downloader.download("Animal",limit=12,output_dir='images',adult_filter_off=True)

In [ ]:
downloader.download("Bird",limit=12,output_dir='images',adult_filter_off=True)

In [ ]:
downloader.download("Insect",limit=12,output_dir='images',adult_filter_off=True)

In [ ]:
#How we create the dataset for this 3 categories


In [ ]:
import numpy as np 

a = np.array([[1,2,3,4],  #Creating the matrix
              [4,5,6,8]])
a.ndim

In [ ]:
# How do we convert Matrix to Vector? - Using the flatten() method
a.flatten()

In [ ]:
# Preprocessing data - 
# 1.Resizing the images
# 2. Flattening
import os #to play around folders
import pandas as pd
import matplotlib.pyplot as plt # Displaying the image
import numpy as np # use for numerical computing
from skimage.io import imread # use opencv uses for reading the image
from skimage.transform import resize # for resizing the image

target = []
images = []     #in the form of 2D i.e. Matrix
flat_data = [] #in the form of vector

datadir = '/content/images'   # we gave it to the path
categories = ['Animal','Bird','Insect'] #here we mention all the categories

for i in categories:
  path = os.path.join(datadir,i)
  #print(path)
  for img in os.listdir(path):
    print(img)
    img_array = imread(os.path.join(path,img))
    #print(img_array)
    img_resized = resize(img_array,(150,150,3)) # Value gets converted 0-1 from 0-255
    flat_data.append((img_resized).flatten())
    images.append(img_resized)
    target.append(i)
    plt.imshow(img_resized)
    plt.show()

flat_data = np.array(flat_data)
target = np.array(target)
images = np.array(images)


In [ ]:
print(img_array) #it shows the matrix

In [ ]:
print(img_array.shape) # it shows he height width and 2d or 1d image.

In [ ]:
len(flat_data[0])

In [ ]:
150*150

In [ ]:
target

In [ ]:
unique,count = np.unique(target,return_counts=True)
plt.bar(categories,count)

In [ ]:
# SPlit Data into Training and Testing
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(flat_data,target,
                                                 test_size=0.3,random_state=109)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm
param_grid = [
              {'C':[1,10,100,1000],'kernel':['linear']},
              {'C':[1,10,100,1000],'gamma':[0.001,0.0001],'kernel':['rbf']},
]

svc = svm.SVC(probability=True)
clf = GridSearchCV(svc,param_grid)
clf.fit(x_train,y_train)

In [ ]:
y_pred = clf.predict(x_test)
y_pred

In [ ]:
y_test

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix


In [ ]:
accuracy_score(y_pred,y_test)

In [ ]:
confusion_matrix(y_pred,y_test)

In [ ]:
# Save the model using Pickel library
import pickle
pickle.dump(clf,open('img_model.p','wb'))

In [ ]:
model = pickle.load(open('img_model.p','rb'))

In [ ]:
# Testing the brand new Image
flat_data = []
url = input('Enter your URL')
img = imread(url)
img_resized = resize(img,(150,150,3))
flat_data.append(img_resized.flatten())
flat_data = np.array(flat_data)
print(img.shape)
plt.imshow(img_resized)
y_out = model.predict(flat_data)
y_out = categories[y_out[0]]
print(f' PREDICTED OUTPUT: {y_out}')

In [ ]:
!pip nstall streamlit

!pip install pyngrok
from pyngrok import ngrok

In [ ]:
#Deployment :- Final and important stage 
# 1. Webpage - HTML/CSS/JS -  Flask/Jango
# 2. Webapp - Streamlit/Dash
# 3. Mobile App - Kotlin/Java


In [ ]:
%%writefile app.py
import streamlit as st
import numpy as np
from skimage.io import imread
from skimage.transfrom import resize
import pickle
from PIL import Image
st.title('Image Classifier Using Machine Learning')
st.text('Upload the Image')

model = pickle.load(open('img_model.p','rb'))

uploaded_file = st.file_uploader("Choose an image...",type="jpg")
if uploaded_file is not None:
  img = Image.open(uploaded_file)
  st.image(img,caption='Uploaded Image')

  if st.button('PREDICT'):
    CATEGORIES = ['Animal','Bird','Insect']
    st.write('Result..')
    flat_data=[]
    img = np.array(img)
    img_resized = resize(img,(150,150,3))
    flat_data.append(img_resized.flatten())
    flat_data = np.array(flat_data)
    print(img.shape)
    plt.imshow(img_resized)
    y_out = model.predict(flat_data)
    y_out = categories[y_out[0]]
    print(f' PREDICTED OUTPUT: {y_out}')



In [ ]:
!nohup streamlit run app.py &

url = ngrok.connect(port='8501')
url